In [1]:
from pyspark import SparkConf, SparkContext
from itertools import combinations
from collections import Counter
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))
pair_counter = Counter()
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = text.lower()
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
#wiki = wiki.map(lambda word :(word ,1)).reduceByKey(lambda x,y: x+y)
result = wiki.take(1)[0]

In [2]:
def collect_pairs(lines):
    pair_counter = Counter()
    unique_tokens = sorted(set(lines))  # exclude duplicates in same line and sort to ensure one word is always before other
    combos = combinations(unique_tokens, 2)
    pair_counter += Counter(combos)
    return pair_counter

In [3]:
def find_bigrams(input_list):
    bigram_dict = {}
    for i in range(len(input_list)-1):
        if 'narodnaya' in (input_list[i], input_list[i+1])[0]:
            return ((str(input_list[i]) +"_" + str(input_list[i+1])),1)

In [4]:
test = wiki.map(find_bigrams)
test_2 = test.filter(lambda x: x != None)
test_3 = test_2.reduceByKey(lambda v1,v2:v1 +v2)

In [5]:
for data in test_3.take(10000):
    print(data[0]+" "+ str(data[1]))
    

narodnaya_gazeta 1
narodnaya_volya 3
